# Sreality.com (real estate marketplace) scrape, clean, transform, load, and Visualise

## Project Purpose:

To demonstrate a full Web scraping ETL project from scrapping through transformation to visualisation

## Business Case:

- To accurately identify apartments that have above average price to rent ratio across streets, districts, and cities

- To accurately identify apartments with below average rents across apartment types and cities

- To accurately identify apartments with below average prices across apartment types and cities

In [1]:
#importing python libraries
from bs4 import BeautifulSoup
import requests as rq

In [2]:
import pandas as pd

## Extracting all tables from the HTML file


Since rent and sales data each are needed for two cities (Brno and Prague), we will be scraping these data 
separately, then merge and concat them. These cities will be selected on the website and data will be extracted
from the url 

In [3]:
#Fetch HTML using request
url = 'https://www.sreality.cz/en/search/to-rent/apartments/jihomoravsky-kraj?page=2'
#Extract HTMl tree
page = rq.get(url).text
soup = BeautifulSoup(page)
properties_list=soup.find('property-list')
print(properties_list)

None


In [4]:
#since Beautiful soup is returning none, we will try selenium
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

### Extracting sales data for Brno

In [ ]:
#sales data brno
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list_sales_brno = 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page='
urls_sales_brno=[]
page_checking = True
page_count = 1
#looping through pages until the last page
#we will maually check the last page to extract the HTML class of every last page 
while page_checking:
    url_sales_brno = url_list_sales_brno + str(page_count)
    driver.get(url_sales_brno)
    driver.maximize_window()
    
    # pause to stay within the limit of number of requests
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")#class of the last page
    
    if last_page is not None:#checking page status
        page_checking = False
    
    else:
        urls_sales_brno.append(url_sales_brno)
        # go to the next page
        page_count = page_count+1
driver.quit()

In [12]:
#looping through each page/url to extract listings
listings_sales_brno=[]
for webpages in urls_sales_brno:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_sales_brno.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields

#### Looping through to get individual properties/fields


In [13]:
#extracting and cleaning apartment types 
types_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[1][:5].strip()
        types_brno_sales.append(apt_type)

In [14]:
#extracting and cleaning apartment sizes without units
sizes_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1].strip()
        sizes_brno_sales.append(apt_size)

In [15]:
#extracting and cleaning units 
units_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1].strip()
        #.split(',')\
        [-1][3:]
        units_brno_sales.append(apt_units)

In [16]:
#extracting and cleaning streets
streets_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(' ')[0].split(',')[0].strip()
        streets_brno_sales.append(apt_street)

In [17]:
#extracting and cleaning municipal_district
municipal_district_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(' ')[1].strip()
        municipal_district_brno_sales.append(apt_municipal_district)

In [18]:
#extracting and cleaning features
other_features_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','').strip()
            else:
                otherfeatures = None
            other_features_brno_sales.append(otherfeatures)

In [19]:
#extracting and cleaning neighbourhood
neighbourhood_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1].strip()
        neighbourhood_brno_sales.append(apt_neighbourhood)

In [20]:
#extracting and cleaning price
price_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','').strip()
        price_brno_sales.append(apt_price)

In [172]:
# create a dataframe that will store all properties 
brno_sales=pd.DataFrame({'types': types_brno_sales,'sizes':sizes_brno_sales,'units':units_brno_sales,'streets'\
                         :streets_brno_sales,'municipal_district':municipal_district_brno_sales,'neighbourhood'\
                         :neighbourhood_brno_sales,'price':price_brno_sales,'other_features':\
                         other_features_brno_sales})

In [161]:
brno_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,1+1,25,m²,Gallašova,Brno,Štýřice,2240000,State owenship,89600.0,1,brno_sales
1,3+1,78,m²,Zikova,Brno,Líšeň,6380000,After reconstruction,81795.0,1,brno_sales
2,3+1,120,m²,Solniční,Brno,Brno-město,9100000,State owenship,75833.0,1,brno_sales
3,2+1,64,m²,Purkyňova,Brno,Brno,4750000,None,74219.0,1,brno_sales
4,2+1,96,m²,Ruská,Brno,Pole,7790000,None,81146.0,1,brno_sales


#### Performing EDA and Dropping outlier rows

In [162]:
brno_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,1+1,25,m²,Gallašova,Brno,Štýřice,2240000,State owenship,89600.0,1,brno_sales
1,3+1,78,m²,Zikova,Brno,Líšeň,6380000,After reconstruction,81795.0,1,brno_sales
2,3+1,120,m²,Solniční,Brno,Brno-město,9100000,State owenship,75833.0,1,brno_sales
3,2+1,64,m²,Purkyňova,Brno,Brno,4750000,None,74219.0,1,brno_sales
4,2+1,96,m²,Ruská,Brno,Pole,7790000,None,81146.0,1,brno_sales


In [24]:
brno_sales['types'].value_counts()

2+kt    210
3+kt    154
1+kt    121
2+1     114
3+1     106
4+kt     64
1+1      48
4+1      25
unus     12
5+kt      9
5+1       4
6 ro      3
Name: types, dtype: int64

In [163]:
brno_sales['sizes'].value_counts()

60     24
55     19
70     18
35     17
66     15
       ..
148     1
287     1
156     1
104     1
180     1
Name: sizes, Length: 137, dtype: int64

In [164]:
brno_sales['municipal_district'].value_counts()

Brno         552
district      36
Vyhlídce,     12
Kasáren,       8
náměstí,       7
            ... 
IV.,           1
Telnice        1
Drásov         1
Dráhy,         1
nábřeží,       1
Name: municipal_district, Length: 66, dtype: int64

In [165]:
brno_sales['streets'].value_counts()

Kamechy        26
Bystrcká       20
Táborská       19
Na             17
Spolková       15
               ..
Královka        1
Podstránská     1
Přírodní        1
Pavlovská       1
Jircháře        1
Name: streets, Length: 339, dtype: int64

In [166]:
brno_sales['neighbourhood'].value_counts()

Brno           65
Židenice       65
Zábrdovice     56
Pole           46
Brno-venkov    40
               ..
Drásov          1
Telnice         1
Popůvky         1
Kounice         1
Střelice        1
Name: neighbourhood, Length: 65, dtype: int64

In [170]:
brno_sales['other_features'].value_counts().head()

Loggia                  77
New buildings           64
After reconstruction    63
In construction         51
Concrete                39
Name: other_features, dtype: int64

In [173]:
brno_sales.drop(brno_sales[brno_sales['price'].str.contains('Information')].index, inplace=True)

In [31]:
brno_sales.drop(brno_sales[brno_sales['streets']=='Brno'].index, inplace=True)

In [32]:
brno_sales['types']=brno_sales['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

#### Checking and converting types

In [33]:
#checking types
print(brno_sales.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [34]:
#converting sizes to int
brno_sales['sizes']=brno_sales['sizes'].astype(int)

In [35]:
#converting prices to int
#prices contains xa0 (unicode character)
brno_sales['price']=brno_sales['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
brno_sales['price']=pd.to_numeric(brno_sales['price'])

In [36]:
brno_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,1+1,25,m²,Gallašova,Brno,Štýřice,2240000,State owenship
1,3+1,78,m²,Zikova,Brno,Líšeň,6380000,After reconstruction
2,3+1,120,m²,Solniční,Brno,Brno-město,9100000,State owenship
3,2+1,64,m²,Purkyňova,Brno,Brno,4750000,None
4,2+1,96,m²,Ruská,Brno,Pole,7790000,None


In [37]:
brno_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 869
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   types               745 non-null    object
 1   sizes               745 non-null    int64 
 2   units               745 non-null    object
 3   streets             745 non-null    object
 4   municipal_district  745 non-null    object
 5   neighbourhood       745 non-null    object
 6   price               745 non-null    int64 
 7   other_features      580 non-null    object
dtypes: int64(2), object(6)
memory usage: 52.4+ KB


In [38]:
brno_sales.describe(include='all')

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
count,745,745.000000,745,745,745,745,7.450000e+02,580
unique,12,NaN,1,339,66,65,NaN,40
top,2+kt,NaN,m²,Kamechy,Brno,Brno,NaN,Loggia
freq,183,NaN,745,26,552,65,NaN,77
mean,NaN,69.252349,NaN,NaN,NaN,NaN,6.197304e+06,NaN
std,NaN,35.856767,NaN,NaN,NaN,NaN,2.959686e+06,NaN
min,NaN,19.000000,NaN,NaN,NaN,NaN,1.260000e+06,NaN
25%,NaN,46.000000,NaN,NaN,NaN,NaN,4.290000e+06,NaN
50%,NaN,61.000000,NaN,NaN,NaN,NaN,5.450000e+06,NaN
75%,NaN,81.000000,NaN,NaN,NaN,NaN,7.250000e+06,NaN


#### Feature Engineering

In [39]:
#Calculating CostPerSquaremetre
brno_sales['CostPerSquaremetre']= brno_sales['price']/brno_sales['sizes']

In [40]:
#rounding up to the nearest whole number
brno_sales['CostPerSquaremetre']=brno_sales['CostPerSquaremetre'].round(0)

In [41]:
#does this apartment contain separate kitchen?
def kitchen(x):
    if 'kt' in  x:
        return '0'
    else:
        return '1'

In [42]:
brno_sales['Separate_kitchen']=brno_sales['types'].apply(kitchen)

In [43]:
#defining data category
brno_sales['data category']= 'brno_sales'

In [44]:
brno_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,1+1,25,m²,Gallašova,Brno,Štýřice,2240000,State owenship,89600.0,1,brno_sales
1,3+1,78,m²,Zikova,Brno,Líšeň,6380000,After reconstruction,81795.0,1,brno_sales
2,3+1,120,m²,Solniční,Brno,Brno-město,9100000,State owenship,75833.0,1,brno_sales
3,2+1,64,m²,Purkyňova,Brno,Brno,4750000,None,74219.0,1,brno_sales
4,2+1,96,m²,Ruská,Brno,Pole,7790000,None,81146.0,1,brno_sales


In [45]:
brno_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 869
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   types               745 non-null    object 
 1   sizes               745 non-null    int64  
 2   units               745 non-null    object 
 3   streets             745 non-null    object 
 4   municipal_district  745 non-null    object 
 5   neighbourhood       745 non-null    object 
 6   price               745 non-null    int64  
 7   other_features      580 non-null    object 
 8   CostPerSquaremetre  745 non-null    float64
 9   Separate_kitchen    745 non-null    object 
 10  data category       745 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 69.8+ KB


### Extracting sales data for Prague

In [ ]:
#scraping sales data for prague
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list = 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page='
urls=[]
page_checking = True
page_count = 1
#looping through pages until the last page
while page_checking:
    url = url_list + str(page_count)
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")
    
    if last_page is not None:
        page_checking = False
    
    else:
        urls.append(url)
        page_count = page_count+1
driver.quit()

In [49]:
#looping through each page to extract listings
listings_sales_prague=[]
for webpages in urls:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    #listingss = soup.find_all('div')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_sales_prague.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields

#### Looping through to get individual properties/fields


In [50]:
#extracting and cleaning  apartment types 
types_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[1][:5]
        types_prague_sales.append(apt_type)

In [51]:
#extracting and cleaning apartment sizes 
sizes_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1]
        sizes_prague_sales.append(apt_size)        

In [52]:
#extracting and cleaning units 
units_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1]
        [-1][3:]
        units_prague_sales.append(apt_units)

In [53]:
#extracting and cleaning streets
streets_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(',',1)[0]
        streets_prague_sales.append(apt_street)

In [54]:
#extracting and cleaning municipal_district
municipal_district_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(',',1)[-1].split('-')[0]
        municipal_district_prague_sales.append(apt_municipal_district.strip())
        

In [55]:
#extracting and cleaning features
other_features_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','')
            else:
                otherfeatures = None
            other_features_prague_sales.append(otherfeatures)

In [56]:
#extracting and cleaning neighbourhood
neighbourhood_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1]
        neighbourhood_prague_sales.append(apt_neighbourhood)

In [57]:
#extracting and cleaning price
price_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','')
        price_prague_sales.append(apt_price)


In [58]:
prague_sales=pd.DataFrame({'types': types_prague_sales,'sizes':sizes_prague_sales,'units':units_prague_sales\
                           ,'streets':streets_prague_sales,'municipal_district':municipal_district_prague_sales,\
                           'neighbourhood':neighbourhood_prague_sales,'price':price_prague_sales,\
                           'other_features':other_features_prague_sales})
prague_sales

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+1,85,m²,Janáčkovo nábřeží,Praha 5,Smíchov,13 900 000,Furnished
1,3+kt,84,m²,Pod Harfou,Praha 9,Vysočany,11 500 000,Parking
2,3+kt,92,m²,Fikerova,Praha 4,Modřany,11 850 000,New buildings
3,2+1,46,m²,Frýdlantská,Praha 8,Kobylisy,5 299 000,Loggia
4,4+kt,118,m²,Moravská,Praha 2,Vinohrady,23 944 513,Not furnished
...,...,...,...,...,...,...,...,...
2836,2+1,66,m²,Máchova,Praha 2,Vinohrady,8 900 000,None
2837,3+kt,70,m²,Kateřinské náměstí,Praha,Průhonic,8 599 500,None
2838,3+kt,70,m²,Praha 4 - Újezd u Průhonic,Praha 4,Průhonic,7 999 865,None
2839,4+kt,110,m²,Nad závodištěm,Praha 5,Chuchle,14 216 000,None


#### Performing EDA and Dropping outlier rows


In [59]:
print(prague_sales['types'].value_counts().to_string())

 2+kt    849
 3+kt    597
 1+kt    371
 3+1     338
 2+1     243
 4+kt    222
 1+1      79
 4+1      68
 5+kt     32
 5+1      16
 6 ro     15
 unus     11


In [174]:
prague_sales['sizes'].value_counts().head()

49    64
52    59
53    58
50    56
70    53
Name: sizes, dtype: int64

In [175]:
prague_sales['municipal_district'].value_counts().head()

Praha 5     447
Praha 10    440
Praha 4     420
Praha 9     377
Praha 8     199
Name: municipal_district, dtype: int64

In [177]:
prague_sales['streets'].value_counts().head()

K Jezeru    93
Zimova      39
Plzeňská    22
Armády      18
Koněvova    16
Name: streets, dtype: int64

In [176]:
prague_sales['neighbourhood'].value_counts().head()

Žižkov      144
Hostivař    126
Stodůlky    121
Smíchov     110
Město       101
Name: neighbourhood, dtype: int64

In [64]:
prague_sales['units'].value_counts()

m²    2841
Name: units, dtype: int64

In [65]:
prague_sales.drop(prague_sales[prague_sales['price'].str.contains('Information')].index, inplace=True)

In [66]:
prague_sales['types']=prague_sales['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

In [67]:
prague_sales.drop(prague_sales[prague_sales['streets']=='Praha'].index, inplace=True)

#### Checking and converting types

In [68]:
#checking types
print(prague_sales.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [69]:
#converting sizes to int
prague_sales['sizes']=prague_sales['sizes'].astype(int)

In [70]:
#converting prices to int
#prices contains xa0 (unicode character)
prague_sales['price']=prague_sales['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
prague_sales['price']=pd.to_numeric(prague_sales['price'])

In [71]:
prague_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+1,85,m²,Janáčkovo nábřeží,Praha 5,Smíchov,13900000,Furnished
1,3+kt,84,m²,Pod Harfou,Praha 9,Vysočany,11500000,Parking
2,3+kt,92,m²,Fikerova,Praha 4,Modřany,11850000,New buildings
3,2+1,46,m²,Frýdlantská,Praha 8,Kobylisy,5299000,Loggia
4,4+kt,118,m²,Moravská,Praha 2,Vinohrady,23944513,Not furnished


In [72]:
prague_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2714 entries, 0 to 2840
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   types               2714 non-null   object
 1   sizes               2714 non-null   int64 
 2   units               2714 non-null   object
 3   streets             2714 non-null   object
 4   municipal_district  2714 non-null   object
 5   neighbourhood       2714 non-null   object
 6   price               2714 non-null   int64 
 7   other_features      2233 non-null   object
dtypes: int64(2), object(6)
memory usage: 190.8+ KB


In [73]:
prague_sales.describe(include='all')

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
count,2714,2714.000000,2714,2714,2714,2714,2.714000e+03,2233
unique,12,NaN,1,1244,11,104,NaN,66
top,2+kt,NaN,m²,K Jezeru,Praha 5,Žižkov,NaN,After reconstruction
freq,812,NaN,2714,93,447,144,NaN,248
mean,NaN,69.436625,NaN,NaN,NaN,NaN,8.499901e+06,NaN
std,NaN,33.612926,NaN,NaN,NaN,NaN,4.808253e+06,NaN
min,NaN,11.000000,NaN,NaN,NaN,NaN,2.280000e+05,NaN
25%,NaN,48.000000,NaN,NaN,NaN,NaN,5.490000e+06,NaN
50%,NaN,63.000000,NaN,NaN,NaN,NaN,6.995000e+06,NaN
75%,NaN,82.000000,NaN,NaN,NaN,NaN,9.993538e+06,NaN


#### Feature Engineering

In [74]:
#Calculating CostPerSquaremetre
prague_sales['CostPerSquaremetre']=prague_sales['price']/prague_sales['sizes']

In [75]:
prague_sales['CostPerSquaremetre']=prague_sales['CostPerSquaremetre'].round(0)

In [76]:
#does this apartment contain separate kitchen?
def kitchen(x):
    if 'kt' in  x:
        return '0'
    else:
        return '1'

In [77]:
#does this apartment contain separate kitchen?
prague_sales['Separate_kitchen']=prague_sales['types'].apply(kitchen)

In [78]:
#defining data category
prague_sales['data category']= 'prague_sales'

In [79]:
prague_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,2+1,85,m²,Janáčkovo nábřeží,Praha 5,Smíchov,13900000,Furnished,163529.0,1,prague_sales
1,3+kt,84,m²,Pod Harfou,Praha 9,Vysočany,11500000,Parking,136905.0,0,prague_sales
2,3+kt,92,m²,Fikerova,Praha 4,Modřany,11850000,New buildings,128804.0,0,prague_sales
3,2+1,46,m²,Frýdlantská,Praha 8,Kobylisy,5299000,Loggia,115196.0,1,prague_sales
4,4+kt,118,m²,Moravská,Praha 2,Vinohrady,23944513,Not furnished,202920.0,0,prague_sales


### Extracting rent data for Brno

In [ ]:
#scraping rent data for Brno
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list_rent_brno = 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page='
urls_rent_brno=[]
page_checking = True
page_count = 1
#looping through pages until the last page
while page_checking:
    url_rent_brno = url_list_rent_brno + str(page_count)
    driver.get(url_rent_brno)
    driver.maximize_window()
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")
    
    if last_page is not None:
        page_checking = False
    
    else:
        urls_rent_brno.append(url_rent_brno)
        page_count = page_count+1
driver.quit()

In [83]:
#looping through each page to extract listings
listings_rent_brno=[]
for webpages in urls_rent_brno:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    #listingss = soup.find_all('div')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_rent_brno.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields

#### Looping through to get individual properties/fields


In [84]:
#extracting and cleaning  apartment types 
types_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[-1][:5]
        types_brno_rent.append(apt_type)

In [85]:
#extracting and cleaning apartment sizes 
sizes_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1]
        sizes_brno_rent.append(apt_size)

In [86]:
#extracting and cleaning units 
units_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1]
        [-1][3:]
        units_brno_rent.append(apt_units)

In [87]:
#extracting and cleaning streets
streets_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(',',1)[0]
        streets_brno_rent.append(apt_street)

In [88]:
#extracting and cleaning municipal_district
municipal_district_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(',',1)[-1].split('-')[0]
        municipal_district_brno_rent.append(apt_municipal_district)

In [89]:
#extracting and cleaning features
other_features_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','')
            else:
                otherfeatures = None
            other_features_brno_rent.append(otherfeatures)

In [90]:
#extracting and cleaning neighbourhood
neighbourhood_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1]
        neighbourhood_brno_rent.append(apt_neighbourhood)

In [91]:
#extracting and cleaning price
price_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','')
        price_brno_rent.append(apt_price)

In [178]:
brno_rent=pd.DataFrame({'types': types_brno_rent,'sizes':sizes_brno_rent,'units':units_brno_rent,'streets'\
                         :streets_brno_rent,'municipal_district':municipal_district_brno_rent,'neighbourhood'\
                         :neighbourhood_brno_rent,'price':price_brno_rent,'other_features':other_features_brno_rent})
brno_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,3+1,80,m²,Teyschlova,Brno,Bystrc,19 000,Loggia
1,1+kt,23,m²,Údolní,Brno,Veveří,11 500,Furnished
2,2+kt,45,m²,Kociánka,Brno,Pole,14,Collective owenship
3,4+kt,83,m²,Ponětovická,Brno,Slatina,30 000,Terrace
4,2+kt,51,m²,Trnitá,Brno,Trnitá,17 000,New buildings


#### Performing EDA and Dropping outlier rows

In [93]:
brno_rent['types'].value_counts()

 1+kt    140
 2+kt    120
 2+1      78
 1+1      57
 3+kt     43
 3+1      36
 4+1      12
 4+kt      8
To re      7
 5+kt      2
 unus      1
 5+1       1
Name: types, dtype: int64

In [179]:
brno_rent['sizes'].value_counts().head()

 30     18
 45     13
 32     11
 24     10
 40      9
Name: sizes, dtype: int64

In [180]:
brno_rent['streets'].value_counts().head()

Bratislavská       11
Václavská          11
Palackého třída     8
Holzova             8
Hybešova            8
Name: streets, dtype: int64

In [181]:
brno_rent['municipal_district'].head()

0     Brno 
1     Brno 
2     Brno 
3     Brno 
4     Brno 
Name: municipal_district, dtype: object

In [182]:
brno_rent['neighbourhood'].value_counts().head()

Pole          51
Brno          51
Veveří        40
Zábrdovice    39
Brno-město    30
Name: neighbourhood, dtype: int64

In [98]:
brno_rent.drop(brno_rent[brno_rent['price'].str.contains('Information')].index, inplace=True)

In [99]:
brno_rent.drop(brno_rent[brno_rent['sizes'].str.contains('room')].index, inplace=True)

In [100]:
brno_rent['types']=brno_rent['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

#### Checking and converting types

In [101]:
#checking types
print(brno_rent.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [102]:
#converting sizes to int
brno_rent['sizes']=brno_rent['sizes'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
brno_rent['sizes']=pd.to_numeric(brno_rent['sizes'])

In [103]:
#converting prices to int
#prices contains xa0 (unicode character)
brno_rent['price']=brno_rent['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
brno_rent['price']=pd.to_numeric(brno_rent['price'])

In [104]:
brno_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,3+1,80,m²,Teyschlova,Brno,Bystrc,19000,Loggia
1,1+kt,23,m²,Údolní,Brno,Veveří,11500,Furnished
2,2+kt,45,m²,Kociánka,Brno,Pole,14,Collective owenship
3,4+kt,83,m²,Ponětovická,Brno,Slatina,30000,Terrace
4,2+kt,51,m²,Trnitá,Brno,Trnitá,17000,New buildings


#### Feature Engineering

In [105]:
#Calculating CostPerSquaremetre
brno_rent['CostPerSquaremetre']= brno_rent['price']/brno_rent['sizes']

In [106]:
brno_rent['CostPerSquaremetre']=brno_rent['CostPerSquaremetre'].round(0)

In [107]:
#does this apartment contain separate kitchen?
brno_rent['Separate_kitchen']=brno_rent['types'].apply(kitchen)

In [108]:
#defining data category
brno_rent['data category']= 'brno_rent'

In [109]:
brno_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,3+1,80,m²,Teyschlova,Brno,Bystrc,19000,Loggia,238.0,1,brno_rent
1,1+kt,23,m²,Údolní,Brno,Veveří,11500,Furnished,500.0,0,brno_rent
2,2+kt,45,m²,Kociánka,Brno,Pole,14,Collective owenship,0.0,0,brno_rent
3,4+kt,83,m²,Ponětovická,Brno,Slatina,30000,Terrace,361.0,0,brno_rent
4,2+kt,51,m²,Trnitá,Brno,Trnitá,17000,New buildings,333.0,0,brno_rent


In [110]:
brno_rent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 504
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   types               495 non-null    object 
 1   sizes               495 non-null    int64  
 2   units               495 non-null    object 
 3   streets             495 non-null    object 
 4   municipal_district  495 non-null    object 
 5   neighbourhood       495 non-null    object 
 6   price               495 non-null    int64  
 7   other_features      385 non-null    object 
 8   CostPerSquaremetre  495 non-null    float64
 9   Separate_kitchen    495 non-null    object 
 10  data category       495 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 46.4+ KB


### Extracting rent data for Prague


In [ ]:
#scraping rent data for Prague
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list_rent_prague = 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page='
urls_rent_prague=[]
page_checking = True
page_count = 1

while page_checking:
    url_rent_prague = url_list_rent_prague + str(page_count)
    driver.get(url_rent_prague)
    driver.maximize_window()
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")
    
    if last_page is not None:
        page_checking = False
    
    else:
        urls_rent_prague.append(url_rent_prague)
        page_count = page_count+1
driver.quit()

In [114]:
listings_rent_prague=[]
for webpages in urls_rent_prague:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    #listingss = soup.find_all('div')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_rent_prague.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields


#### Looping through to get individual properties/fields


In [115]:
#extracting and cleaning  apartment types 
types_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[-1][:5]
        types_prague_rent.append(apt_type)

In [116]:
#extracting and cleaning apartment sizes 
sizes_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1].replace(u'\xa0',u' ')
        sizes_prague_rent.append(apt_size) 

In [117]:
#extracting and cleaning units 
units_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1]
        [-1][3:]
        units_prague_rent.append(apt_units)

In [118]:
#extracting and cleaning streets
streets_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(',',1)[0]
        streets_prague_rent.append(apt_street)

In [119]:
#extracting and cleaning municipal_district
municipal_district_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(',',1)[-1].split('-')[0]
        municipal_district_prague_rent.append(apt_municipal_district.strip())

In [120]:
#extracting and cleaning features
other_features_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','')
            else:
                otherfeatures = None
            other_features_prague_rent.append(otherfeatures)

In [121]:
#extracting and cleaning neighbourhood
neighbourhood_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1]
        neighbourhood_prague_rent.append(apt_neighbourhood)

In [122]:
#extracting and cleaning price
price_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','')
        price_prague_rent.append(apt_price)

In [123]:
prague_rent=pd.DataFrame({'types': types_prague_rent,'sizes':sizes_prague_rent,'units':units_prague_rent\
                           ,'streets':streets_prague_rent,'municipal_district':municipal_district_prague_rent,\
                          'neighbourhood':neighbourhood_prague_rent,'price':price_prague_rent,\
                          'other_features':other_features_prague_rent})
prague_rent

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+kt,50,m²,Pod Vyšehradem,Praha 4,Podolí,21 000,Garage
1,2+1,47,m²,Odborů,Praha 2,Město,24 000,Grocery 5 min walk
2,3+kt,70,m²,Tomáškova,Praha 5,Smíchov,25 000,Cellar
3,2+1,40,m²,Souběžná III,Praha 5,Jinonice,15 000,Terrace
4,3+kt,115,m²,Radimova,Praha 6,Břevnov,36 000,New buildings
...,...,...,...,...,...,...,...,...
1950,3+kt,109,m²,Filmařská,Praha 5,Hlubočepy,33 000,None
1951,3+kt,88,m²,Toužimská,Praha 9,Kbely,19 500,Balcony
1952,3+kt,107,m²,Italská,Praha 2,Vinohrady,61 321,Balcony
1953,5+1,210,m²,Pod Kaštany,Praha 6,Bubeneč,80 000,Cellar


#### Performing EDA and Dropping outlier rows

In [124]:
prague_rent['types'].value_counts()

 2+kt    636
 3+kt    325
 1+kt    321
 2+1     169
 3+1     162
 1+1     136
 4+kt     87
 4+1      47
To re     25
 5+kt     16
 5+1      15
 unus     15
 6 ro      1
Name: types, dtype: int64

In [184]:
prague_rent['sizes'].value_counts().head()

50    67
45    55
40    53
55    52
60    45
Name: sizes, dtype: int64

In [185]:
prague_rent['streets'].value_counts().head()

Italská      31
Máchova      22
Holečkova    22
Laubova      21
Sázavská     17
Name: streets, dtype: int64

In [127]:
prague_rent['municipal_district'].value_counts()

Praha 5     290
Praha 4     252
Praha 2     232
Praha 10    187
Praha 1     186
Praha 3     184
Praha 6     151
Praha 9     151
Praha 8     128
Praha       113
Praha 7      81
Name: municipal_district, dtype: int64

In [186]:
prague_rent['neighbourhood'].value_counts().head()

Vinohrady    255
Město        199
Žižkov       117
Smíchov      114
Libeň         66
Name: neighbourhood, dtype: int64

In [187]:
prague_rent['other_features'].value_counts().head()

New buildings            243
After reconstruction     241
Balcony                  119
Terrace                  103
Partly funished           95
Name: other_features, dtype: int64

In [130]:
prague_rent.drop(prague_rent[prague_rent['sizes'].str.contains('r')].index,inplace=True)

In [131]:
prague_rent.drop(prague_rent[prague_rent['price'].str.contains('Information')].index, inplace=True)

In [132]:
prague_rent['types']=prague_rent['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

#### Checking and converting types

In [133]:
#checking types
print(prague_rent.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [134]:
#converting sizes to int
prague_rent['sizes']=prague_rent['sizes'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
prague_rent['sizes']=pd.to_numeric(prague_rent['sizes'])

In [135]:
#converting prices to int
#prices contains xa0 (unicode character)
prague_rent['price']=prague_rent['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
prague_rent['price']=pd.to_numeric(prague_rent['price'])

In [136]:
prague_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+kt,50,m²,Pod Vyšehradem,Praha 4,Podolí,21000,Garage
1,2+1,47,m²,Odborů,Praha 2,Město,24000,Grocery 5 min walk
2,3+kt,70,m²,Tomáškova,Praha 5,Smíchov,25000,Cellar
3,2+1,40,m²,Souběžná III,Praha 5,Jinonice,15000,Terrace
4,3+kt,115,m²,Radimova,Praha 6,Břevnov,36000,New buildings


#### Feature Engineering


In [137]:
#Calculating CostPerSquaremetre
prague_rent['CostPerSquaremetre']= prague_rent['price']/prague_rent['sizes']

In [138]:
prague_rent['CostPerSquaremetre']=prague_rent['CostPerSquaremetre'].round(0)

In [139]:
#does this apartment contain separate kitchen?
prague_rent['Separate_kitchen']=prague_rent['types'].apply(kitchen)

In [140]:
#defining data category
prague_rent['data category']= 'prague_rent'

In [141]:
prague_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,2+kt,50,m²,Pod Vyšehradem,Praha 4,Podolí,21000,Garage,420.0,0,prague_rent
1,2+1,47,m²,Odborů,Praha 2,Město,24000,Grocery 5 min walk,511.0,1,prague_rent
2,3+kt,70,m²,Tomáškova,Praha 5,Smíchov,25000,Cellar,357.0,0,prague_rent
3,2+1,40,m²,Souběžná III,Praha 5,Jinonice,15000,Terrace,375.0,1,prague_rent
4,3+kt,115,m²,Radimova,Praha 6,Břevnov,36000,New buildings,313.0,0,prague_rent


#### Loading data to CSV and importing to Tableau for More visualisation

In [157]:
#unioning files by city
brno_all_data=pd.concat([brno_sales,brno_rent], axis=0).reset_index()

In [158]:
prague_all_data=pd.concat([prague_sales,prague_rent], axis=0).reset_index()

In [159]:
#Loading file 1
brno_all_data.to_excel('brno_all_data.xlsx', index=False)

In [160]:
#Loading file 2
prague_all_data.to_excel('prague_all_data.xlsx', index=False)